<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong><em>Change publicly accessible of RDS DB Snapshots to private</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Secure-Publicly-accessible-Amazon-RDS-Snapshot"><u>Secure Publicly accessible Amazon RDS Snapshot</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p><br>1.&nbsp;Get publicly accessible DB snapshots<br>2.&nbsp;Change the public access to private</p>

In [6]:
if public_snapshot_ids and not region:
    raise SystemExit("Provide a region for the Snapshots!")
if region == None:
    region = ""

<h3 id="Get-publicly-accessible-DB-snapshots"><a id="2" target="_self" rel="nofollow"></a>Get publicly accessible DB snapshots</h3>
<p>Using unSkript's Get Publicly Accessible DB Snapshots in RDS action we will fetch all the publicly accessible snapshots from the list of manual DB snapshots.</p>
<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_snapshots</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.utils import CheckOutput
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.aws.aws_filter_all_manual_database_snapshots.aws_filter_all_manual_database_snapshots import aws_filter_all_manual_database_snapshots


from beartype import beartype
@beartype
def aws_get_publicly_accessible_db_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_publicly_accessible_db_snapshots(handle, region: str=None) -> Tuple:
    """aws_get_publicly_accessible_db_snapshots lists of publicly accessible
       db_snapshot_identifier.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the RDS.

        :rtype: Object with status, result having publicly accessible Snapshots
        Identifier in RDS, error
    """
    manual_snapshots_list = []
    result = []
    all_regions = [region]
    if region is None or not region:
        all_regions = aws_list_all_regions(handle=handle)
    try:
        for r in all_regions:
            snapshots_dict = {}
            output = aws_filter_all_manual_database_snapshots(handle=handle, region=r)
            snapshots_dict["region"] = r
            snapshots_dict["snapshot"] = output
            manual_snapshots_list.append(snapshots_dict)
    except Exception as e:
        raise e

    for all_snapshots in manual_snapshots_list:
        try:
            ec2Client = handle.client('rds', region_name=all_snapshots['region'])
            for each_snapshot in all_snapshots['snapshot']:
                response = ec2Client.describe_db_snapshot_attributes(
                    DBSnapshotIdentifier=each_snapshot
                    )
                db_attribute = response["DBSnapshotAttributesResult"]
                for value in db_attribute['DBSnapshotAttributes']:
                    p_dict={}
                    if "all" in value["AttributeValues"]:
                        p_dict["region"] = all_snapshots['region']
                        p_dict["open_snapshot"] = db_attribute['DBSnapshotIdentifier']
                        result.append(p_dict)
        except Exception:
            pass
    if len(result)!=0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not public_snapshot_ids",
    "condition_result": true
    }''')

task.configure(outputName="publicly_accessible_snapshots")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_publicly_accessible_db_snapshots, lego_printer=aws_get_publicly_accessible_db_snapshots_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Public-DB-Snapshots">Create List of Public DB Snapshots</h3>
<p>This action filters regions that have no manual DB snapshots and creates a list those that have</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output:&nbsp;<code>all_public_snapshots</code></p>
</blockquote>

In [5]:
all_public_snapshots = []
try:
    if publicly_accessible_snapshots[0] == False:
        for snapshot in publicly_accessible_snapshots[1]:
            all_public_snapshots.append(snapshot)
except Exception as e:
    if public_snapshot_ids:
        for snap in public_snapshot_ids:
            data_dict = {}
            data_dict["region"] = region
            data_dict["open_snapshot"] = snap
            all_public_snapshots.append(data_dict)
    else:
        raise Exception(e)

<h3 id="Change-the-public-access-to-private">Change the public access to private</h3>
<p>Using unSkript's Modify Publicly Accessible RDS Snapshots action we will modify the access to all the publicly accessible snapshots from the <em>public</em> to <em>private</em>.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code>, <code>db_snapshot_identifier</code></p>
</blockquote>

In [6]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_modify_public_db_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_modify_public_db_snapshots(handle, db_snapshot_identifier: str, region: str) -> List:
    """aws_modify_public_db_snapshots lists of publicly accessible DB Snapshot Idntifier Info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type db_snapshot_identifier: string
        :param db_snapshot_identifier: DB Snapshot Idntifier of RDS.

        :type region: string
        :param region: Region of the RDS.

        :rtype: List with Dict of DB Snapshot Idntifier Info.
    """


    ec2Client = handle.client('rds', region_name=region)
    result = []
    try:
        response = ec2Client.modify_db_snapshot_attribute(
            DBSnapshotIdentifier=db_snapshot_identifier,
            AttributeName='restore',
            ValuesToRemove=['all'])

        result.append(response)

    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "db_snapshot_identifier": "iter.get(\\"open_snapshot\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_public_snapshots",
    "iter_parameter": ["region","db_snapshot_identifier"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_public_snapshots) != 0",
    "condition_result": true
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_modify_public_db_snapshots, lego_printer=aws_modify_public_db_snapshots_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to secure all the publicly accessible AWS RDS DB Snapshots by using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>